## リクエストデータの仕様

以下のような JSON を期待値計算ソフト (サーバー側) にポート 8888 で投げる。

```json
{
    "version": "0.9.0",
    "zikaze": 27,
    "bakaze": 27,
    "turn": 3,
    "syanten_type": 1,
    "dora_indicators": [27],
    "flag": 63,
    "hand_tiles": [1,1,1,4,5,6,11,12,20,20,23,23,24,30],
    "melded_blocks": [],
    "counts":[4,1,4,4,3,3,3,4,4,4,4,3,3,4,4,4,4,4,4,4,2,4,4,2,3,4,4,3,4,4,3,4,4,4,1,1,1]
}

```

* 牌は0~36の数値で表す (リクエストデータ、レスポンスデータ共通)
  * 0 ~ 8: 萬子1~ 9
  * 9 ~ 17: 筒子1~ 9
  * 18 ~ 26: 索子1~ 9
  * 27: 東
  * 28: 南
  * 29: 西
  * 30: 北
  * 31: 白
  * 32: 發
  * 33: 中
  * 34: 赤五萬
  * 35: 赤五筒
  * 36: 赤五索

* version: 現在の mahjong-cpp のバージョンは 0.9.0 です。バージョンが一致していることの確認に使用します。
* zikaze: 自風牌 (東: 27, 南:28, 西:29, 北:30)
* bakaze: 場風牌 (東: 27, 南:28, 西:29, 北:30)
* turn: 現在の巡目 (1 ~ 17)
* syanten_type: (一般手:1, 七対子手:2, 国士無双手:4)
* dora_indicators: ドラ表示牌の一覧 (槓ドラ含む)
* flag: 計算の設定フラグ (ビットフラグなので、OR で複数指定可)
  * ビットフラグなので、OR で複数指定可 (例: 向聴落とし考慮、手変わり考慮のみ有効なら 1 + 2 = 3 を指定)
  * 1: 向聴落とし考慮
  * 2: 手変わり考慮
  * 4: ダブル立直考慮
  * 8: 一発考慮
  * 16: 海底撈月考慮
  * 32: 裏ドラ考慮
  * 64: 和了確率を最大化 (指定されていない場合は期待値を最大化)
* hand_tiles: 手牌を13枚または14枚指定
* counts: 37 種類の各牌の残り枚数(場に見えていない枚数)です。基本的には手牌とドラ表示牌を引いておけばよいですが、さらに河など場に見えている枚数を考慮したい場合はその牌の枚数を引いておきます。
    * 例: 手牌 [1,1,1,4,5,6,11,12,20,20,23,23,24,30], ドラ表示牌: [27] の場合、`counts` は以下のようになります。
    * `[4,1,4,4,3,3,3,4,4,4,4,3,3,4,4,4,4,4,4,4,2,4,4,2,3,4,4,3,4,4,3,4,4,4,1,1,1]`

## レスポンスデータの仕様

正しいリクエストを投げると、返ってきた JSON の `response/candidates` キーに各牌の有効牌、期待値、和了確率、聴牌確率が格納されています。


* tile: 打牌する牌
* required_tiles
  * count: 残り枚数
  * tile: 有効牌
* syanten_down: 向聴戻しになるかどうか
* exp_values: 各巡目における期待値
* win_probs: 各巡目における和了確率
* tenpai_probs: 各巡目における聴牌確率

```json
{
    "tile": 1,
    "required_tiles": [
        {"count": 3, "tile": 1},
        {"count": 4, "tile": 4},
        {"count": 3, "tile": 16},
        {"count": 3, "tile": 25},
    ],
    "syanten_down": True,
    "exp_values": [
        3891.424884520901,
        3611.387861422344,
        3324.9602806561834,
        3032.9968246078943,
        2736.61314257078,
        2437.2401686922626,
        2136.6891411450547,
        1837.2294593562674,
        1541.681958625137,
        1253.5307198421215,
        977.0571881068047,
        717.5011748665181,
        481.2542973307469,
        276.0926081180646,
        111.45663941547303,
        0.0,
        0.0,
    ],
    "win_probs": [
        0.28676429520079144,
        0.2648826688812986,
        0.2425884467324838,
        0.21996018286576025,
        0.19709834604910853,
        0.17412977831069812,
        0.15121302315346807,
        0.12854469610673472,
        0.10636710599982613,
        0.08497737871415958,
        0.06473838800697322,
        0.046091862473938344,
        0.029574116525271518,
        0.015834949744313175,
        0.005660377358490566,
        0.0,
        0.0,
    ],
    "tenpai_probs": [
        0.86033118739783,
        0.8433620793246688,
        0.8241517682984493,
        0.802380082468734,
        0.7776775927773254,
        0.7496174831278617,
        0.7177059858794518,
        0.6813711127748264,
        0.6399493574355539,
        0.5926699801291114,
        0.5386364060646058,
        0.47680417182584156,
        0.40595473676059113,
        0.32466433231730357,
        0.23126684636118597,
        0.1238095238095238,
        0.0,
    ],
}
```

## requests モジュールを使った例

1. 期待値計算ソフトを起動する。
2. 以下のようにしてリクエストを投げる。

In [ ]:
import requests
import json
from pprint import pprint

req_data = {
    "version": "0.9.0",
    "zikaze": 27,
    "bakaze": 27,
    "turn": 3,
    "syanten_type": 1,
    "dora_indicators": [27],
    "flag": 63,
    "hand_tiles": [1,1,1,4,5,6,11,12,20,20,23,23,24,30],
    "melded_blocks": [],
    "counts":[4,1,4,4,3,3,3,4,4,4,4,3,3,4,4,4,4,4,4,4,2,4,4,2,3,4,4,3,4,4,3,4,4,4,1,1,1]
}

# dict -> json
payload = json.dumps(req_data)

# リクエストを送信する。
res = requests.post(
    "http://localhost:8888", payload, headers={"Content-Type": "application/json"}
)
res_data = res.json()
pprint(res_data)